In [1]:
import pandas as pd
import gcsfs
import xarray as xr
from datetime import datetime
import itertools

import myconfig
from mytasks import Check, Download, ReadFiles, SaveAsZarr, Upload, Cleanup
from mysearch import esgf_search

In [16]:
# CONFIGURE ESGF Search here
node_pref = myconfig.node_pref
dtype = myconfig.dtype
myconfig.local_target_prefix = '/h114/naomi/zarr-minimal/'

# reset the preference rank to omit a particular data node
# node_pref['esgf-data1.llnl.gov'] = 999

ESGF_site = dtype['llnl']
#ESGF_site = dtype['dkrz']

print('zarrs will be written to: ',myconfig.local_target_prefix)

zarrs will be written to:  /h114/naomi/zarr-minimal/


In [3]:
# CONFIGURE GCS
fs     = gcsfs.GCSFileSystem(token='anon', access='read_only',cache_timeout=-1)
df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')

# make available to all modules
myconfig.fs = fs
myconfig.df_GCS = df_GCS

In [4]:
core_experiments = [
    '1pctCO2', 'abrupt-4xCO2',  'historical', 'piControl' 
    ,'ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585'
                   ]
more_experiments = [
     'piControl-spinup', 'amip-hist', 'esm-hist', 'esm-piControl', 'esm-piControl-spinup'
    ,'1pctCO2-bgc','lgm', 'past1000', 'amip'
                   ]
                   
core_Amon_2dvars = ['evspsbl', 'hfls', 'pr', 'prc', 'ps', 'psl', 'sfcWind', 'tas', 'ts', 'uas', 'vas','huss','hurs']
flux_Amon_2dvars = ['rlds', 'rlus', 'rsds', 'rsus', 'hfds', 'hfls', 'hfss','tauu','tauv']
core_Omon_2dvars = ['tos', 'sos', 'zos']
flux_Omon_2dvars = ['tauuo', 'tauuo']
core_Amon_3dvars = ['ta', 'ua', 'va', 'zg', 'wap', 'hur', 'hus']
core_Omon_3dvars = ['masscello', 'so', 'thetao', 'umo', 'uo', 'vmo', 'vo', 'wmo', 'wo']
core_Omon_tracers = ['chl', 'chlos', 'dfe', 'dfeos', 'epc100', 'fgco2', 'intpp', 'no3', 'no3os', 'phyc', 'phycos', 'phydiat', 'phydiatos', 'si', 'sios', 'spco2', 'zooc', 'zoocos']

In [5]:
# Pick keyword values to specify your search here. Not specifying a particular keyword means it will find all.
all_search = {
     'table_id'      : ['Omon','Amon','AERmon','SImon']
    ,'experiment_id' : ['historical','ssp585','ssp370']
    ,'variable_id'   : ["uas", "vas", "chl", "clt", "sithick", "siconc", "sisnthick", "sisnconc","tas","toz"]
    #,'member_id'     : ['r1i1p1f1']
    #,'source_id'     : ['CESM2-WACCM']
}

#search = 'O2d-1'
search = 'hiba'
search = 'trond'

# define some common searches:
if search == 'O2d-1f':
    all_search = {'table_id': ['Omon'], 'experiment_id': core_experiments, 'variable_id': flux_Omon_2dvars}
    label = f'{tab1}-core_experiments-flux_Omon_2dvars'
if search == 'O2d-1c':
    all_search = {'table_id': ['Omon'], 'experiment_id': core_experiments, 'variable_id': core_Omon_2dvars}
    label = f'{tab1}-core_experiments-core_Omon_2dvars'
if search == 'hiba':
    all_search = {'table_id': ['day'], 'experiment_id': ['ssp370','ssp126','ssp585','ssp245','historical'], 
                  'variable_id': ['tas','pr','tasmax','tasmin','vas','uas']}
    label = 'hiba'
if search == 'trond':
    all_search = {'table_id': ['Omon'], 'experiment_id': ['ssp370','ssp585','historical'], 
                  'variable_id': ["thetao","o2"]}
    label = 'trond'


all_search

{'table_id': ['day'],
 'experiment_id': ['ssp370', 'ssp126', 'ssp585', 'ssp245', 'historical'],
 'variable_id': ['tas', 'pr', 'tasmax', 'tasmin', 'vas', 'uas']}

In [6]:
update_ESGF = False
if update_ESGF:
    x = [value for key,value in all_search.items()]
    searches = [p for p in itertools.product(*x)]

    dESGF = []
    for s in searches:
        search = dict(zip(all_search.keys(),s))
        print(search)
        df = esgf_search(search, server=ESGF_site)
        if len(df)>0:
            dESGF += [df]

    df_ESGF = pd.concat(dESGF)
    df_ESGF.to_csv(f'csv/ESGF_{label}.csv',index=False)
else:
    df_ESGF = pd.read_csv(f'csv/ESGF_{label}.csv', dtype='unicode')

len(df_ESGF), len(df_ESGF.ds_dir.unique())

(129720, 7007)

In [7]:
# make df of all needed
NewNeeded = False
if NewNeeded:
    df = pd.merge(df_ESGF,df_GCS, how='outer', indicator=True)
    df_needed = df[df._merge == 'left_only']
    
    keep_keys = df_ESGF.keys()
    all_keys = df.keys()
    drop_keys = list(set(all_keys) - set(keep_keys))
    df_needed = df_needed.drop(drop_keys,1)

    num_stores = 0
    if len(df_needed) > 0:
        num_stores = df_needed.ds_dir.nunique() 
        print(f'needed: nfiles={len(df_needed)}, nstores={num_stores}')
    else:
        print('no new data available')
        exit    
    
    df_needed['member'] = [int(s.split('r')[-1].split('i')[0]) for s in df_needed['member_id']]
    df_needed['zsize'] = [df_needed[df_needed.ds_dir==zs]['file_size'].sum() for zs in df_needed['ds_dir']]
    df_needed = df_needed.sort_values(by=['zsize'])
    
    df_needed.to_csv(f'csv/needed_{label}.csv',index=False)
else:
    df_needed = pd.read_csv(f'csv/needed_{label}.csv')

len(df_needed), len(df_needed.ds_dir.unique())

(20241, 265)

In [8]:
# make available to all modules
myconfig.df_needed = df_needed

In [9]:
ds_dirs = df_needed.ds_dir.unique()
numdsets = len(ds_dirs)

In [17]:
label = 'more'
request_id = 'test' #datetime.now().strftime('%Y%m%d-%H%M')
progress_log  = f'logs/progress_{label}.log'
failure_log  = f'logs/failure_{label}.log'
success_log  = f'logs/success_{label}.log'

In [18]:
def write_log(file,str,verbose=True):
    f = open(file,'a')
    if verbose:
        print(str)
    f.write(f'{str}\n')
    f.close()
    return

In [ ]:
# reload the catalog
df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')

# refresh the gcsfs
fs.invalidate_cache()

for item, ds_dir in enumerate(ds_dirs):
    
    print(f'\n{item}/{numdsets-1}',ds_dir)
    #if 'EC-Earth3' in ds_dir:
    #    continue
    #if 'IITM-ESM' in ds_dir:
    #    continue
    #if '/uas' not in ds_dir:
    #    continue

    (ierr, exc) = Check(ds_dir)
    if ierr > 0:
        write_log(progress_log,f'{ds_dir} {ierr}: {exc}'); continue
        
    (gfiles, ierr, exc) = Download(ds_dir)
    if ierr > 0:
        write_log(failure_log,f'{ds_dir},noUse, {ierr}: {exc}'); continue
        
    (ds,ierr,exc) = ReadFiles(ds_dir, gfiles)
    if ierr > 0:
        write_log(failure_log,f'{ds_dir},noUse, {ierr}: {exc}'); continue

    (ierr,exc) = SaveAsZarr(ds_dir, ds)
    if ierr > 0:
        write_log(failure_log,f'{ds_dir},noUse, {ierr}: {exc}'); continue
        
    (gsurl, ierr,exc) = Upload(ds_dir)
    if ierr > 0:
        write_log(failure_log,f'{ds_dir},noUse, {ierr}: {exc}'); continue

    (ierr,exc) = Cleanup(ds_dir, gfiles)
    if ierr > 0:
        write_log(failure_log,f'{ds_dir},noUse, {ierr}: {exc}'); continue

    write_log(success_log,f'{item}/{numdsets-1}: {ds_dir} saved to {gsurl}'); continue


0/264 ScenarioMIP/NCAR/CESM2-WACCM/ssp370/r3i1p1f1/day/tas/gn
ScenarioMIP/NCAR/CESM2-WACCM/ssp370/r3i1p1f1/day/tas/gn 1: noUse in codes

1/264 ScenarioMIP/NIMS-KMA/KACE-1-0-G/ssp370/r1i1p1f1/day/vas/gr
netcdfs/vas_day_KACE-1-0-G_ssp370_r1i1p1f1_gr_20150101-21001230.nc
85 86
successfully uploaded as gs://cmip6/ScenarioMIP/NIMS-KMA/KACE-1-0-G/ssp370/r1i1p1f1/day/vas/gr
1/264: ScenarioMIP/NIMS-KMA/KACE-1-0-G/ssp370/r1i1p1f1/day/vas/gr saved to gs://cmip6/ScenarioMIP/NIMS-KMA/KACE-1-0-G/ssp370/r1i1p1f1/day/vas/gr

2/264 ScenarioMIP/CAS/FGOALS-g3/ssp126/r2i1p1f1/day/pr/gn
netcdfs/pr_day_FGOALS-g3_ssp126_r2i1p1f1_gn_20150101-20151231.nc
ScenarioMIP/CAS/FGOALS-g3/ssp126/r2i1p1f1/day/pr/gn,noUse, 2: Server not responding for: http://esgf-data3.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/ScenarioMIP/CAS/FGOALS-g3/ssp126/r2i1p1f1/day/pr/gn/v20200923/pr_day_FGOALS-g3_ssp126_r2i1p1f1_gn_20150101-20151231.nc

3/264 ScenarioMIP/CAS/FGOALS-g3/ssp126/r4i1p1f1/day/pr/gn
netcdfs/pr_day_FGOALS-g3_s